In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

TRAIN_PATH = "dataset.csv"
MODEL_A_PATH = "modelA_penultimate.pth"
MODEL_B_PATH = "modelB_penultimate.pth"
SUBMISSION_PATH = "submission.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ITERATIONS = 100

class PenultimateModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.net(x)

df = pd.read_csv(TRAIN_PATH)
x_cols = [c for c in df.columns if c.startswith("x")]
y_cols = [c for c in df.columns if c.startswith("y")]
IDs = df["ID"].values

X = df[x_cols].values.astype(np.float32)
Y = df[y_cols].values.astype(np.float32)

print(f"Loaded data: X={X.shape}, Y={Y.shape}")

scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)
scaler_Y = StandardScaler().fit(Y)
Y_scaled = scaler_Y.transform(Y)

stateA = torch.load(MODEL_A_PATH, map_location="cpu")
stateB = torch.load(MODEL_B_PATH, map_location="cpu")

modelA = PenultimateModel()
modelA.load_state_dict(stateA)
modelA.to(DEVICE).eval()

modelB = PenultimateModel()
modelB.load_state_dict(stateB)
modelB.to(DEVICE).eval()

XA = torch.from_numpy(X_scaled).to(DEVICE)
with torch.no_grad():
    ZA = modelA(XA).cpu().numpy()
    ZB = modelB(XA).cpu().numpy()

print("Embeddings shapes:", ZA.shape, ZB.shape)

def fit_linear(Z, Y):
    W, _, _, _ = np.linalg.lstsq(Z, Y, rcond=None)
    return W
def fit_linear_ridge(Z, Y, alpha=1e-3):
    model = Ridge(alpha=alpha, fit_intercept=True)
    model.fit(Z, Y)
    return model

def reconstruction_error(Y, Z, W):
    return np.linalg.norm(Y - Z @ W, axis=1)

#preds = np.random.choice(["A", "B"], size=len(X))
errA_init = np.linalg.norm(Y_scaled - ZA @ np.linalg.pinv(ZA) @ Y_scaled, axis=1)
errB_init = np.linalg.norm(Y_scaled - ZB @ np.linalg.pinv(ZB) @ Y_scaled, axis=1)
preds = np.where(errA_init < errB_init, "A", "B")



for it in range(ITERATIONS):
    maskA = preds == "A"
    maskB = preds == "B"

    W_A = fit_linear(ZA[maskA], Y_scaled[maskA]) if maskA.any() else fit_linear(ZA, Y_scaled)
    W_B = fit_linear(ZB[maskB], Y_scaled[maskB]) if maskB.any() else fit_linear(ZB, Y_scaled)

    errA = reconstruction_error(Y_scaled, ZA, W_A)
    errB = reconstruction_error(Y_scaled, ZB, W_B)

    #preds_new = np.where(errA < errB, "A", "B")
    probA = np.exp(-errA) / (np.exp(-errA) + np.exp(-errB))
    preds_new = np.where(probA > 0.45, "A", "B")


    if np.array_equal(preds, preds_new):
        print(f"Converged at iteration {it}")
        break
    preds = preds_new

submission = pd.DataFrame({"ID": IDs, "Source": preds})
submission.to_csv(SUBMISSION_PATH, index=False)
print(f"Saved predictions to {SUBMISSION_PATH}")
print(submission.head())


Loaded data: X=(5000, 100), Y=(5000, 5)
Embeddings shapes: (5000, 10) (5000, 10)
Converged at iteration 7
✅ Saved EM-style predictions to submission.csv
   ID Source
0   1      A
1   2      B
2   3      B
3   4      B
4   5      A
